# Test 1: Basic Text Extraction with Layout Parser

**Goal:** Verify Google Document AI Layout Parser works and extracts raw text from PDFs

**What this test does:**
- Connects to Google Document AI with Layout Parser processor
- Processes a PDF document
- Extracts text using `document_layout.blocks` (Layout Parser API)
- Saves results to JSON file

**Layout Parser API (Dec 2025):**
- Returns data in `document.document_layout.blocks` (NOT `document.pages`)
- Each block contains `text_block.text` and `text_block.type_`
- Block types: paragraph, heading, list_item, table, etc.
- Superior structure detection for academic documents


## Step 1: Install Dependencies

Run this cell to install required packages for Google Colab.


In [ ]:
%pip install -q google-cloud-documentai python-dotenv openai anthropic pdf2image Pillow
print("✅ All dependencies installed!")


: 

## Step 2: Upload Credentials

Upload your Google Cloud service account JSON file.


In [ ]:
from google.colab import files
import json
import os

print("📤 Please upload your Google Cloud credentials JSON file...")
uploaded = files.upload()

# Save credentials
creds_filename = list(uploaded.keys())[0]
credentials_content = json.loads(uploaded[creds_filename].decode('utf-8'))

# Write to temporary file
with open('docai-credentials.json', 'w') as f:
    json.dump(credentials_content, f)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'docai-credentials.json'
print(f"✅ Credentials saved: {creds_filename}")


## Step 3: Configure Environment

Set your Google Cloud project ID and Layout Parser processor ID.


In [ ]:
# Configuration - UPDATE THESE VALUES
DOCAI_PROJECT_ID = "your-project-id-here"  # Replace with your project ID
DOCAI_PROCESSOR_ID = "your-processor-id-here"  # Replace with your Layout Parser processor ID
DOCAI_LOCATION = "us"  # us, eu, or asia

# Set environment variables
os.environ['DOCAI_PROJECT_ID'] = DOCAI_PROJECT_ID
os.environ['DOCAI_PROCESSOR_ID'] = DOCAI_PROCESSOR_ID
os.environ['DOCAI_LOCATION'] = DOCAI_LOCATION

print(f"✅ Configuration set:")
print(f"   Project: {DOCAI_PROJECT_ID}")
print(f"   Processor: {DOCAI_PROCESSOR_ID}")
print(f"   Location: {DOCAI_LOCATION}")


## Step 4: Initialize Document AI Client

Set up the Document AI client for Layout Parser.


In [ ]:
# Initialize Document AI client directly (Layout Parser API - Dec 2025)
from google.cloud import documentai_v1 as documentai

# Build processor name
processor_name = f"projects/{DOCAI_PROJECT_ID}/locations/{DOCAI_LOCATION}/processors/{DOCAI_PROCESSOR_ID}"

# Create client
client = documentai.DocumentProcessorServiceClient()

print(f"✅ Document AI client initialized")
print(f"   Processor: {processor_name}")


## Step 5: Verify Setup

Test connection to Google Document AI.


In [ ]:
print("🔍 Verifying Document AI setup...\n")

try:
    # Get processor info to verify it exists and is enabled
    processor = client.get_processor(name=processor_name)
    
    print(f"✅ Processor found!")
    print(f"   Name: {processor.display_name}")
    print(f"   Type: {processor.type_}")
    print(f"   State: {documentai.Processor.State(processor.state).name}")
    
    if processor.state == documentai.Processor.State.ENABLED:
        print("\n✅ Setup verified! Ready to process documents.")
    else:
        print(f"\n⚠️ Processor state is {documentai.Processor.State(processor.state).name}")
        
except Exception as e:
    print(f"\n❌ Setup verification failed: {e}")
    print("\nPlease check:")
    print("  1. Credentials file is valid")
    print("  2. Project ID is correct")
    print("  3. Processor ID is correct")
    print("  4. Layout Parser processor exists and is enabled")


## Step 6: Upload PDF for Testing

Upload your PDF file to process.


In [ ]:
print("📤 Please upload your PDF file...")
uploaded_pdfs = files.upload()

# Get the first uploaded PDF
pdf_filename = list(uploaded_pdfs.keys())[0]
pdf_path = pdf_filename

print(f"✅ PDF uploaded: {pdf_filename}")
print(f"   Size: {len(uploaded_pdfs[pdf_filename]) / 1024:.1f} KB")


## Step 7: Process Document with Layout Parser

Extract text from the PDF using Layout Parser processor.


In [ ]:
print(f"{'='*60}")
print("TEST 1: BASIC TEXT EXTRACTION (Layout Parser API)")
print(f"{'='*60}\n")

# Read PDF content
with open(pdf_path, 'rb') as f:
    pdf_content = f.read()

print(f"📄 Processing PDF with Layout Parser: {pdf_path}")
print(f"   File size: {len(pdf_content):,} bytes")

# Create process request
request = documentai.ProcessRequest(
    name=processor_name,
    raw_document=documentai.RawDocument(
        content=pdf_content,
        mime_type="application/pdf"
    ),
    skip_human_review=True
)

# Process document
result = client.process_document(request=request)
document = result.document

# Layout Parser returns data in document_layout.blocks (NOT document.pages/text)
doc_layout = document.document_layout

# Extract all text from blocks
full_text = ""
total_blocks = 0

if doc_layout and doc_layout.blocks:
    total_blocks = len(doc_layout.blocks)
    for block in doc_layout.blocks:
        if block.text_block and block.text_block.text:
            full_text += block.text_block.text + "\n"

print(f"✅ Document processed successfully!")
print(f"   Total blocks: {total_blocks}")
print(f"   Total characters: {len(full_text):,}")


## Step 8: Extract Blocks by Type

Extract text blocks organized by type (paragraph, heading, list, table, etc.).


In [ ]:
print("📝 Extracting blocks by type...\n")

blocks_data = []
block_types = {}

if doc_layout and doc_layout.blocks:
    for block in doc_layout.blocks:
        block_info = {
            "block_id": block.block_id,
            "type": None,
            "text": None,
            "page_start": None,
            "page_end": None
        }
        
        # Get text block info
        if block.text_block:
            block_info["type"] = str(block.text_block.type_) if block.text_block.type_ else "unknown"
            block_info["text"] = block.text_block.text if block.text_block.text else ""
            
            # Count block types
            block_type = block_info["type"]
            block_types[block_type] = block_types.get(block_type, 0) + 1
        
        # Get page span
        if block.page_span:
            block_info["page_start"] = block.page_span.page_start
            block_info["page_end"] = block.page_span.page_end
        
        blocks_data.append(block_info)

# Print summary
print(f"📊 Block Types Found:")
for block_type, count in sorted(block_types.items()):
    print(f"   {block_type}: {count}")

print(f"\n✅ Extracted {len(blocks_data)} blocks")


## Step 9: Preview Extracted Text

Display a preview of the extracted text.


In [ ]:
print("📄 TEXT PREVIEW (first 800 characters):")
print("=" * 60)
print(full_text[:800])
if len(full_text) > 800:
    print("...")
print("=" * 60)


## Step 10: Save Results to JSON

Save extraction results to a JSON file.


In [ ]:
import json
from pathlib import Path

# Build results
results = {
    "pdf_file": Path(pdf_path).name,
    "total_blocks": len(blocks_data),
    "total_characters": len(full_text),
    "block_type_counts": block_types,
    "full_text": full_text,
    "blocks": blocks_data
}

# Save to JSON
output_path = "test1_raw_text.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"💾 Results saved to: {output_path}")
print(f"\n📊 Summary:")
print(f"  Total blocks: {len(blocks_data)}")
print(f"  Total characters: {len(full_text):,}")
print(f"  Block types: {block_types}")


## Step 11: Download Results

Download the JSON results file to your computer.


In [ ]:
from google.colab import files

print("📥 Downloading results...")
files.download(output_path)
print(f"✅ Test 1 complete! Results downloaded.")
